In [1]:
#@title License information { display-mode: "form" }
#@markdown Copyright 2024 The MediaPipe Authors.
#@markdown Licensed under the Apache License, Version 2.0 (the "License");
#@markdown
#@markdown you may not use this file except in compliance with the License.
#@markdown You may obtain a copy of the License at
#@markdown
#@markdown https://www.apache.org/licenses/LICENSE-2.0
#@markdown
#@markdown Unless required by applicable law or agreed to in writing, software
#@markdown distributed under the License is distributed on an "AS IS" BASIS,
#@markdown WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#@markdown See the License for the specific language governing permissions and
#@markdown limitations under the License.

In [2]:
#@title Setup { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
install_out = widgets.Output()
display(install_out)
with install_out:
  !pip install mediapipe
  from mediapipe.tasks.python.genai import bundler

install_out.clear_output()
with install_out:
  print("Setup done.")

Setup done.


# Why do we need task bundles?

Executing a text generation pipeline in its entirety requires more than merely employing the core transformer model. It requires preparing the input text to align with the model's required format, running the model autoregressively, and sampling during each iteration. Consequently, once the user has converted their model into `tflite` format (comprising the model's graph and parameters), it becomes essential to augment it with additional metadata to ensure successful end-to-end execution of the model.

Task bundler offers a practical approach to creating such bundles. The example below illustrates how a task bundle can be generated for a converted Gemma model.

In [3]:
tflite_model="distilbert-qa.tflite" # @param {type:"string"}
tokenizer_model="PATH/tokenizer.model" # @param {type:"string"}
start_token="<bos>" # @param {type:"string"}
stop_token="<eos>" # @param {type:"string"}
output_filename="PATH/gemma.task" # @param {type:"string"}
enable_bytes_to_unicode_mapping=False # @param ["False", "True"] {type:"raw"}

config = bundler.BundleConfig(
    tflite_model=tflite_model,
    tokenizer_model=tokenizer_model,
    start_token=start_token,
    stop_tokens=[stop_token],
    output_filename=output_filename,
    enable_bytes_to_unicode_mapping=enable_bytes_to_unicode_mapping,
)
bundler.create_bundle(config)

NameError: name 'bundler' is not defined

**Notes:**

* The current task pipeline only supports SentencePiece tokenizer models.
* Certain models (e.g., phi-2) use bytes to unicode mapping. Use `enable_bytes_to_unicode_mapping` flag accordingly. Such information, including `start_token` and `stop_tokens` are often provided along with model artifacts.
* The generated output bundle file must end with `.task`. If not, `create_bundle` automatically adds the extension. Do not remove or change that.

In [ ]:
#@title Download the Bundle { display-mode: "form" }
#@markdown Run this cell to download the generated `.task` file.
files.download(output_filename)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
tokenizer.save_pretrained("bert_tokenizer/")


RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
No module named 'numpy.rec'

In [ ]:
# build_fake_corpus.py
with open("bert_tokenizer/vocab.txt", "r") as fin, open("fake_corpus.txt", "w") as fout:
    for line in fin:
        token = line.strip()
        if token and not token.startswith("["):
            fout.write(token.replace("##", "") + "\n")


In [ ]:
!pip install sentencepiece

# Train SentencePiece tokenizer
!spm_train \
  --input=fake_corpus.txt \
  --model_prefix=distilbert_tokenizer \
  --vocab_size=30522 \
  --character_coverage=1.0 \
  --model_type=word

In [ ]:
!find / -name "spm_train" 2>/dev/null

In [ ]:
!apt-get update && apt-get install -y cmake build-essential pkg-config libgoogle-perftools-dev
!git clone https://github.com/google/sentencepiece.git
%cd sentencepiece
!mkdir build && cd build && cmake .. && make -j $(nproc) && make install && ldconfig
%cd /content

In [ ]:
!spm_train --version

In [ ]:
!spm_train \
      --input=fake_corpus.txt \
      --model_prefix=distilbert_tokenizer \
      --vocab_size=26598 \
      --character_coverage=1.0 \
      --model_type=word

In [ ]:
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
model.save("distilbert_tf_model")

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("distilbert_tf_model")
tflite_model = converter.convert()


#tflite_model.save()

# Save the model to a file
with open("distilbert-qa.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
#!pip install --upgrade --force-reinstall mediapipe-model-maker==0.2.1.4
!pip install --upgrade  --user mediapipe-model-maker==0.2.1.4
!python3 -m mediapipe_model_maker.genai.task_converter \
  --tflite_model distilbert-qa.tflite \
  --tokenizer_model distilbert_tokenizer.model \
  --output_file distilbert_qa.task

In [ ]:
pip show mediapipe-model-maker

In [ ]:
!pip install tensorflow text
!pip install tensorflow_hub

In [ ]:
import tensorflow as tf; print(tf.__version__)